# Import Libraries

In [2]:
import os
import sys
import rasterio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
from pathlib import Path
import geopandas as gpd

import rasterio.mask
from rasterio.mask import mask


# Import Constants

In [3]:
# Get the current working directory
current_dir = os.path.abspath('')

# Search for the 'constants.py' file starting from the current directory and moving up the hierarchy
project_root = current_dir
while not os.path.isfile(os.path.join(project_root, 'constants.py')):
    project_root = os.path.dirname(project_root)

# Add the project root to the Python path
sys.path.append(project_root)


In [13]:
from constants import MODIFIED_RASTER, GRUPO_RASTER, DATA_PATH, CROPPED_HANSEN_TEN_MASK, HANSEN_TREECOVER_FILEPATH, SOIL, DISSOLVED_CLEAN_YEAR, TREECOVER_10_AND_ABOVE


In [14]:
# Where files will save, can add subfolders if desired
output_dir = os.path.join(DATA_PATH, 'ml_data')
os.makedirs(output_dir, exist_ok=True)

# TREECOVER MASK

In [6]:
with rasterio.open(HANSEN_TREECOVER_FILEPATH) as src:
    tree_cover_data = src.read(1)
    tree_cover_meta = src.meta

# Create a mask where tree cover is less than 10%
low_tree_cover_mask = tree_cover_data < 10


# Reapply Mask to Hansen deforestation

In [6]:
with rasterio.open(MODIFIED_RASTER) as src:
    final_raster_data = src.read(1)
    final_raster_meta = src.meta

# Apply the low tree cover mask to final_modified_raster0
final_raster_data[low_tree_cover_mask] = -1

# Save the updated raster
updated_raster_path = os.path.join(output_dir, 'updated_final_modified_raster.tif')
with rasterio.open(updated_raster_path, 'w', **final_raster_meta) as dst:
    dst.write(final_raster_data, 1)


# CONVERT TO BINARY

In [8]:

raster_path = r"C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\updated_final_modified_raster.tif"

with rasterio.open(raster_path) as src:
    raster_data = src.read(1)
    raster_meta = src.meta


In [9]:
binary_raster_path = os.path.join(output_dir, 'binary_deforestation_raster.tif')

with rasterio.open(raster_path) as src:
    raster_data = src.read(1)
    raster_meta = src.meta

    # Create a mask for nodata values
    nodata_mask = raster_data == -1

    # Convert values from 1-22 to 1, and everything else to 0, except for nodata values
    binary_raster_data = np.where((raster_data >= 1) & (raster_data <= 22), 1, 0)

    # Reapply the nodata mask to retain -1 values for nodata areas
    binary_raster_data[nodata_mask] = -1

    # Save the binary raster
    with rasterio.open(binary_raster_path, 'w', **raster_meta) as dst:
        dst.write(binary_raster_data, 1)